In [4]:
import pandas as pd
import regex as re

In [5]:
SENTENCES_RE = '[\.\!\?]+'
WORDS_RE = '[^[:alnum:]]+'


def split_with_rexp(text, rsplt):
    sentences = re.split(rsplt, text.lower())
    return (
        sentences
        if sentences[-1] != ''
        else sentences[:-1]
    )


def split_by_sentences(text):
    t = split_with_rexp(text, SENTENCES_RE)
    return t


def split_by_words(text):
    return filter(lambda x: bool(x), split_with_rexp(text, WORDS_RE))


def premorph(text):
    return map(
        split_by_words,
        split_by_sentences(text)
    )

In [6]:
par = pd.read_excel('par_cls.xlsx', encoding='utf-8', header=None)
par.columns = ['id', 'role', 'note', 'end']
fund_dict = pd.read_excel('fund_dict.xlsx', encoding='utf-8', header=None)
fund_dict.columns = ['begin', 'end', 'id']

phrase = 'Кохайтеся, чорнобриві, Та не з москалями, Бо москалі — чужі люди, Роблять лихо з вами. '

In [7]:
# Stage 2 Morpho
words = []
class Word:
    def __init__(self, word, id, begin, ending):
        self.word = word
        self.id = id
        self.begin = begin
        self.ending = ending
        self.role = None
        
    def __str__(self):
        return "{}, {}, {}, {}".format(self.word, self.id, self.ending, self.role)

In [8]:
for i in premorph(phrase):
    for word in list(i):
        for beg in fund_dict['begin']:
            if not word.find(beg):
                words.append(Word(
                        word,
                        int(fund_dict.loc[fund_dict['begin'] == beg]['id']),
                        beg,
                        word[len(beg):] if len(word[len(beg):]) > 0 else ''
                    ))

In [9]:
for word in words:
    for index, row in par.iterrows():
        if row['end'] == word.ending and row['id'] == word.id:
            word.role = row['role']
        elif (type(row['end']) is not str) and row['id'] == word.id and word.ending == '':
            word.role = row['role']

In [11]:
for word in words:
    print(word.word, word.role)

кохайтеся ДЛ
чорнобриві ПЙ
та С
не Ч
з П
москалями ІР
бо П
москалі ІС
чужі ПЙ
люди МВ
лихо ЛШ
з П
вами ЗЕ
